# On Policy Monte-Carlo control

# Importing all the libraries

In [ ]:
from env import Grid_World
import matplotlib.pyplot as plt 
import numpy as np
import random
from statistics import mean
import time
from IPython.display import clear_output
from IPython import display


# Creating the environment 

In [ ]:
envsize = 4
Render_mode = "rgb_array"
env = Grid_World(size= envsize, type="fixed", render_mode=Render_mode)

In [ ]:
agent,target = env.reset()
if env.render_mode == "rgb_array":
    frame = env.render()
    plt.imshow(frame)
    plt.axis = ('off')
    display.display(plt.gcf())
elif env.render_mode == "human":
    env.py_init()
    env.render()
    time.sleep(5)
    env.close()
    

# Creating the Q(s|a) value table

In [ ]:
action_values = np.random.rand(envsize, envsize, 4)

### Testing the value with state (0,0)

In [ ]:
ap = action_values[0,0]
np.argmax(ap)

# Defining the policy

In [ ]:
def policy(state, epsilon = 0.1):
    action_probablities =  action_values[state[0], state[1]]
    if random.uniform(0,1) < epsilon:
        # Choosing a random action 
        action = np.random.choice(4)
    else:
        # Choosing action according to the q-values
        action = np.argmax(action_probablities) 
    return action


### Testing the policy with position (0,0)

In [ ]:
action_values[0,0]

# Implementing the algorithm

In [ ]:
env.render_mode = "rgb_array"

In [ ]:
def on_policy_mc_control(policy: callable, action_values, episodes: int = 100, gamma: int = 0.9, epsilon: int = 0.1) -> None:
    
    sa_returns = np.empty(shape=(envsize, envsize, 4), dtype= object)

    for episode in range(0, episodes + 1):
        trajectory = []
        state, _ = env.reset()
        done = False
        while not done:
            action = policy(state, epsilon)
            next_state, reward, done, _ = env.step(action)
            trajectory.append([state, action, reward])
            state = next_state

        G = 0

        for state_t, action_t, reward_t in reversed(trajectory):
            G = reward_t + gamma * G
            if sa_returns[state_t[0],state_t[1], action_t] == None: 
                sa_returns[state_t[0], state_t[1] ,action_t] = [G]
            else:
                sa_returns[state_t[0],state_t[1], action_t].append(G)
            action_values[state_t[0], state_t[1], action_t] = mean(sa_returns[state_t[0], state_t[1], action_t])


In [ ]:
on_policy_mc_control(policy, action_values)

In [ ]:
print(action_values)

In [ ]:
def test_agent(policy, episodes = 1, epsilon = 0.1):
    env.render_mode = Render_mode
    if env.render_mode == "human":
        env.py_init()
    
    for episode in range(0,episodes):
        state, _= env.reset()
        done = False
        while not done: 
            action = policy(state, epsilon)
            state, _, done, _ = env.step(action)
            frame = env.render()
            if env.render_mode == "rgb_array":
                plt.imshow(frame)
                plt.axis = ('off')
                display.display(plt.gcf())
                display.clear_output(wait = True)
    if env.render_mode == "human":
        env.close()


In [ ]:
test_agent(policy, episodes= 5)